# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [34]:
# your code here
data = pd.read_csv('data/Admission_Predict.csv')
data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [35]:
# your code here
data = data.rename(columns=lambda x: x.strip())

Let's evaluate the dataset by looking at the `head` function.

In [36]:
# your code here
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [37]:
# your code here
data = data[data.notnull()]
data ## same number of rows, no missing data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [38]:
# your code here
data = data.set_index('Serial No.', drop=False)
data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [39]:
# your code here
len(data.apply(lambda x: (x['GRE Score'], x['CGPA']), axis=1).unique())
#returns same number of rows

385

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [41]:
data.set_index(['GRE Score', 'CGPA'], inplace=True)
data.head()

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [43]:
# your code here
data.reset_index(inplace=True)
data

,index,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,4,330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...,...,...
380,380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,382,330,9.45,383,116,4,5.0,4.5,1,0.91
383,383,312,8.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [113]:
# your code here
data = pd.read_csv('data/Admission_Predict.csv')
best = data[(data['CGPA'] > 9) & (data['Research'] > 0)]

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [114]:
# your code here
kinda = data[(data['CGPA'] > 9) & (data['SOP'] < 3.5)]

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [115]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    # your code here
    return pd.Series((col-np.mean(col))/np.std(col))


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [116]:
# your code here

lor_std = standardize(data['LOR '])
cgpa_std = standardize(data['CGPA'])
gre_std = standardize(data['GRE Score'])

data['LOR_std'] = lor_std
data['CGPA_std'] = cgpa_std
data['GRE_std'] = gre_std


We will generate the decision choice at random using the code below. Please run the cell.

In [117]:
# Libraries
from random import choices

In [118]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=data.shape[0])
decision_choice

['GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_s

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [119]:
# your code here
data['deciding_column'] = data.lookup(data.index, decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [120]:
# your code here
data['decision'] = [1 if data > 0.8 else 0 for data in data['deciding_column']]
data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,LOR_std,CGPA_std,GRE_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.193197,1.750174,1.755663,1.755663,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,0.076840,-0.992501,-0.063450,0.076840,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,-1.039517,0.121191,0.456297,0.456297,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.481338,-0.643433,-0.236698,-0.481338,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,-0.481338,1.234884,1.149292,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.076840,0.736216,0.629546,0.736216,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.076840,0.852571,0.716170,0.852571,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.193197,1.417729,1.149292,1.149292,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.635019,0.304036,-0.409947,0.304036,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [121]:
# your code here
data['decision'].sum()

91

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [122]:
# your code here
data = data.rename(columns = lambda x: x.strip().strip().replace(' ', '_').lower())
data

,serial_no.,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,lor_std,cgpa_std,gre_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.193197,1.750174,1.755663,1.755663,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,0.076840,-0.992501,-0.063450,0.076840,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,-1.039517,0.121191,0.456297,0.456297,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.481338,-0.643433,-0.236698,-0.481338,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,-0.481338,1.234884,1.149292,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.076840,0.736216,0.629546,0.736216,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.076840,0.852571,0.716170,0.852571,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.193197,1.417729,1.149292,1.149292,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.635019,0.304036,-0.409947,0.304036,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [124]:
# your code here
data['adjusted_gre'] = pd.cut(data[data['university_rating'] >= 4].apply(
    lambda x: x['gre_score'] + 10, axis=1), bins=4, labels=[1,2,3,4])

data[data['adjusted_gre'].notnull()]

,serial_no.,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,lor_std,cgpa_std,gre_std,deciding_column,decision,adjusted_gre
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.193197,1.750174,1.755663,1.755663,1,4
4,5,330,115,5,4.5,3.0,9.34,1,0.90,-0.481338,1.234884,1.149292,1.149292,1,4
9,10,327,111,4,4.0,4.5,9.00,1,0.84,1.193197,0.669726,0.889419,1.193197,1,3
10,11,328,112,4,4.0,4.5,9.10,1,0.78,1.193197,0.835949,0.976043,1.193197,1,4
18,19,325,114,4,3.0,2.0,8.40,0,0.70,-1.597696,-0.327610,0.716170,-0.327610,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,371,335,117,5,5.0,5.0,9.82,1,0.96,1.751376,2.032753,1.582414,2.032753,1,4
377,378,326,112,4,4.0,3.5,9.12,1,0.84,0.076840,0.869194,0.802794,0.869194,1,3
379,380,329,111,4,4.5,4.0,9.23,1,0.89,0.635019,1.052039,1.062667,1.052039,1,4
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.193197,1.417729,1.149292,1.149292,1,4
